#### Baselines Notbook

This notebook goes over building a linear predictor for the spectral envelope of the next sample.  The model looks like this:

*y*(*n*) = a~1~*y*(*n*-1)-a~2~*y*(*n*-2) ... a~*M*~*y*(*n*-*M*)+*e*(*n*)

* *M* is the order of the linear predictor
* a~*i*=1~^*M*^ is the prediction coeffients
* *e*(*n*) ya basic error term

Its worth noting that we only look at *M* samples in the past to predict our waveform.

Most of the rote linear algebra to solve this equation (and DSP know-how) will be taken care of by audiolazy, a lazy python audio library that has the implementations we need.

(The first part of this notebook goes over building the LPC predictor, the next part is the generator function)

In [ ]:
from audiolazy import lpc, Stream, WavStream, sHz, AudioIO
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import numpy as np

import os
import pickle

In [ ]:
# useful constants
DATA_DIR = "./20-test-dataset"
ARCHIVE_NAME = "20-test-dataset.npy"
LABELED_ARCHIVE_NAME = "20-test-archive.pkl"

In [ ]:
# get the archive into memory
song_data = np.load(ARCHIVE_NAME)
print(song_data.shape)
print(song_data[0].shape)

In [ ]:
# load the pickled archive into memory
labeled_song_data = {}
with open(LABELED_ARCHIVE_NAME, 'rb') as f:
    labeled_song_data = pickle.load(f)
print([key for key in labeled_song_data.keys()])

In [ ]:
# hm, the numpy version has some PRETTY WILD numbers, lets see what loading directly from a wav source looks like
# NOTE: code chunk left in for posterity, we're operating on numpy arrays.  The numbers turned out to be right.
wav_files = os.listdir(DATA_DIR)
test_wav = wav_files[0]
str_data = WavStream(os.path.join(DATA_DIR, test_wav))
str_data.take(10)

In [ ]:
#trying to plot the spectral envelope of order 10 (use the 10 previous samples)
# note: I had to blow out the type to avoid numeric overflow problems (which would probably lead to unstable solutions?)
lpc_coeffs = lpc(song_data[10].astype('float64'), order=10)

In [ ]:
print(lpc_coeffs)

In [ ]:
# lets check out some signal response curves
fig1 = lpc_coeffs.plot(plt.figure())
fig1.show()

Ok, lets try to use the lpc coeffs as a filter to generate sound.
Steps:
* run the LPC as a filter over the input signal, get the residuals
* divide the analysis filter into 1 (the equation we want is 1 / the good shit) to get a synthesis filter
* play the residuals forward using the synthesis filter

In [ ]:
analysis_filt = lpc_coeffs
residual = list(analysis_filt(song_data[9].astype('float64')))
synth_filt = 1 / analysis_filt

# should be the frequency response, or ye normie audio graph.  
# Lets see if we can chart it!
synth_audio = synth_filt(residual).take(1000) #the first 1k samples isn't even a ms of audio
synth_timesteps = list(range(len(synth_audio)))

# convert the 16-bit audio to a signal from -1 to 1 (ye normie way of looking at audio)
plot_audio = [sample / 32768.0 for sample in synth_audio] 
plt.plot(synth_timesteps, plot_audio)
plt.show()

In [ ]:
# that looks painful to listen to.  Lets get an accuracy metric comparing the first 200 samples
true_data = Stream(song_data[9].astype('float64')).take(1000)
mse = mean_squared_error(synth_audio, true_data)
print("MSE: {}".format(mse))

In [ ]:
# be INTENSELY careful with this segment.  LPC reconstructed audio is at FULL VOLUME
# can't tell if this is a feature or a bug.

rate=44100 #standard audio rate
s, Hz = sHz(rate)
gen_audio = synth_filt(residual)
with AudioIO(True) as player: # True means "wait for all sounds to stop"
    clip = gen_audio.take(int(10 * s))
    # DON'T BE WEARING HEADPHONES RIGHT NOW, YOU WILL DIE
    player.play(clip, rate=rate) #COMMENTED OUT FOR SAFETY, UNCOMMENT AT YOUR OWN RISK

In [ ]:
# cell to play a song streight rather than reconstructing
rate=44100 #standard audio rate
s, Hz = sHz(rate)
audio = Stream(song_data[9])
with AudioIO(True) as player:
    clip = audio.take(int(5 * s))
    player.play(clip, rate=rate)

#### USING THE .pkl ARCHIVE

We'd like some more control over which songs we're using to infer the coeffs on the linear function for, and which songs we're using those coeffs to play back.
THIS IS THAT THING.

In [ ]:
# complete the loop using the pickle file to have better control on which files are being used for for LPC stuff
train_song = '<?> - Soccer Boss (song 1).wav'
gen_song = 'Henke <?> - Space Game (song 1).wav'
analysis_filt = lpc(labeled_song_data[train_song]['data'].astype('float64'), order=10)
fig1 = analysis_filt.plot(plt.figure())
fig1.show()

In [ ]:
residual = list(labeled_song_data[gen_song]['data'].astype('float64'))
synth_filt = 1 / analysis_filt

gen_audio = synth_filt(residual)
with AudioIO(True) as player: # True means "wait for all sounds to stop"
    clip = gen_audio.take(int(10 * s))
    # DON'T BE WEARING HEADPHONES RIGHT NOW, YOU WILL DIE
    player.play(clip, rate=rate) #COMMENTED OUT FOR SAFETY, UNCOMMENT AT YOUR OWN RISK

#### packaging up LPC as a prediction generator

The basic idea is:
* infer the prediction coeffs on some set of data
* generate audio for a current song (which may or may not be in the prediction dataset)
* run the predictor forward to get a sample for each timestep
* yeild each sample in turn

In [ ]:
def lpc_prediction_gen(order, pred_song, train_data=None, coeffs=None):
    """ Generator to get lpc predictions for a current song
    Parameters:
    order: int
        Number of coefficents to use for the LPC encoding
    pred_song: numpy array, dtype something that can be coerced to 'float64'
        The song to predict with, we'll use this as our data to get the next prediction.
        If identical to the train_data, we're trying to reconstruct the current song
    train_data: numpy array, dtype something that can be coerced to 'float64'
        This is the training data to infer the LPC coefficents used for prediction
        If identical to pred_song, we're trying to reconstruct the current song
        If using multiple songs, they should be boundried with order samples of silence (usually a 0)
    coeffs: an audiolazy lpc instance (the thing lpc returns in audiolazy)
        Optional.  Pass this in instead of training data if we want to reuse a set of already calculated equations
    """
    if not coeffs:
        print("Building an LP spectral predictor with {:d} coefficents".format(order))
        analysis_filt = lpc(train_data.astype('float64'), order=order)
        print("Filter built, calculating generative filter...")
        synth_filt = 1 / analysis_filt
    if coeffs:
        analysis_filt = coeffs
        synth_filt = 1 / coeffs

    print("Getting residuals for current song...")
    # ok, now lets get the residuals on the prediction song
    residuals = list(analysis_filt(pred_song.astype('float64')))
    
    print("Ready to generate samples!")
    # now we can start to yield new audio batches off the current residuals FOREVER
    while True:
        sample_stream = synth_filt(residual)
        while True:
            try:  # this is bad, but I'm not sure how to get the length of an audio stream
                yield sample_stream.take()
            except:
                break

In [ ]:
lpc_gen = lpc_prediction_gen(3, song_data[0], train_data=song_data[1], coeffs=None)

In [ ]:
next(lpc_gen)

In [ ]:
# check to make sure we loop around
for i in range(song_data[0].shape[0]):
    next(lpc_gen)

In [ ]:
next(lpc_gen)